In [15]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [16]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [17]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [18]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [19]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [20]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [21]:
df['loc']

1     (9001 Collins Ave s 309, Surfside, FL 33154, U...
2     (100 South Pointe Dr, Miami Beach, FL 33139, U...
3     (10203 Collins Ave Unit 2103, Bal Harbour, FL ...
4     (8701 Collins Ave #601, Miami Beach, FL 33154,...
5     (791 Crandon Blvd, Miami, FL 33149, USA, (25.6...
6     (2015 Fisher Island Dr #2015, Miami Beach, FL ...
7     (789 Crandon Blvd APT 1003, Key Biscayne, FL 3...
8     (248 Washington Ave #3, Miami Beach, FL 33139,...
9     (18975 Collins Ave #603, Sunny Isles Beach, FL...
10    (15701 Collins Ave #3004, North Miami Beach, F...
Name: loc, dtype: object

In [22]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [23]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Four Seasons Surf Club\r9001 Collins Ave S-309...,3/25/2021,2/22/2022,290,"$21,250,000.00",9361,"$4,810.96",Dora Puig,Dora Puig,Four Seasons Surf Club 9001 Collins Ave S-309 ...,Four Seasons Surf Club,9001 Collins Ave S-309 Surfside,"(9001 Collins Ave s 309, Surfside, FL 33154, U...","(25.8768949, -80.12146349999999, 0.0)",25.876895,-80.121463,0.0
2,Continuum South\r100 S Pointe Dr 1201/02\rMiam...,12/30/2021,2/23/2022,18,"$4,200,000.00",2226,"$1,886.79",Matias Derito,Matias Derito,Continuum South 100 S Pointe Dr 1201/02 Miami ...,Continuum South,100 S Pointe Dr 1201/02 Miami Beach,"(100 South Pointe Dr, Miami Beach, FL 33139, U...","(25.766765, -80.13283659999999, 0.0)",25.766765,-80.132837,0.0
3,Oceana Bal Harbour\r10203 Collins Ave 2103\rBa...,1/6/2022,2/25/2022,18,"$3,800,000.00",2145,"$1,771.56",Timothy Hartmann,Christian Dreyfuss PA,Oceana Bal Harbour 10203 Collins Ave 2103 Bal ...,Oceana Bal Harbour,10203 Collins Ave 2103 Bal Harbour,"(10203 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
4,Eighty Seven Park\r8701 Collins Ave 601\rMiami...,11/30/2021,2/24/2022,5,"$3,360,000.00",2984,"$1,307.90",Patrick Vysata,Ivan Chorney,Eighty Seven Park 8701 Collins Ave 601 Miami B...,Eighty Seven Park,8701 Collins Ave 601 Miami Beach,"(8701 Collins Ave #601, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
5,Ocean Tower Two Condo\r791 Crandon Blvd PH3\rK...,7/17/2021,2/22/2022,194,"$3,350,000.00",2271,"$1,475.12",Marcelo Violland,Katia Reisler,Ocean Tower Two Condo 791 Crandon Blvd PH3 Key...,Ocean Tower Two Condo,791 Crandon Blvd PH3 Key Biscayne,"(791 Crandon Blvd, Miami, FL 33149, USA, (25.6...","(25.6883371, -80.15793239999999, 0.0)",25.688337,-80.157932,0.0
6,Bayside Village East\r2015 Fisher Island Dr 20...,5/16/2021,2/22/2022,249,"$3,030,000.00",3081,$983.45,Tomas Hoffmann,Stephan Burke,Bayside Village East 2015 Fisher Island Dr 201...,Bayside Village East,2015 Fisher Island Dr 2015 Miami Beach,"(2015 Fisher Island Dr #2015, Miami Beach, FL ...","(25.7582172, -80.14165059999999, 0.0)",25.758217,-80.141651,0.0
7,Club Tower One Condo\r789 Crandon Blvd 1003\rK...,12/27/2021,2/25/2022,13,"$2,710,000.00",1443,"$1,878.03",Juan Alvarez,Michelle Gunter,Club Tower One Condo 789 Crandon Blvd 1003 Key...,Club Tower One Condo,789 Crandon Blvd 1003 Key Biscayne,"(789 Crandon Blvd APT 1003, Key Biscayne, FL 3...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
8,El Cyclon Condo\r248 Washington Ave 3\rMiami B...,9/16/2021,2/22/2022,121,"$2,550,000.00",3273,$779.10,Rodrigo Kirilauscas,Kiel Pollitt,El Cyclon Condo 248 Washington Ave 3 Miami Beach,El Cyclon Condo,248 Washington Ave 3 Miami Beach,"(248 Washington Ave #3, Miami Beach, FL 33139,...","(25.7721308, -80.13515989999999, 0.0)",25.772131,-80.135160,0.0
9,Residences by Armani Casa\r18975 Collins Ave 6...,10/1/2021,2/22/2022,103,"$2,485,000.00",2174,"$1,143.05",Jacqueline Gallardo,Lourdes Lorenzo-Luaces,Residences by Armani Casa 18975 Collins Ave 60...,Residences by Armani Casa,18975 Collins Ave 603 Sunny Isles Beach,"(18975 Collins Ave #603, Sunny Isles Beach, FL...","(25.9519943, -80.11997, 0.0)",25.951994,-80.119970,0.0
10,Ritz-Carlton\r15701 Collins Ave 3004\rSunny Is...,9/21/2021,2/23/2022,122,"$2,195,000.00",3068,$819.64,Theresa Maieli,Ken Rubman LLC,Ritz-Carlton 15701 Collins Ave 3004 Sunny Isle...,Ritz-Carlton,15701 Collins Ave 3004 Sunny Isles Beach Beach,"(15701 Collins Ave #3004, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [24]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [25]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [26]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 27th - March 5th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [27]:
m.save('index.html')